In [92]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import re
encoder=LabelEncoder()

#불러오기,결측값제거
df = pd.read_csv('./list 2.csv',sep=',')
df = df.dropna()

#기업명
df["기업명"] = df["기업명"].apply(lambda x: re.sub('[|0-9기]', '', x))      # ‘| 1기’ 와 같은 기수 제외

#관심분야 더미변수화
interests = ['경영','컨설팅','마케팅','경제','금융','과학','공학','기술','IT',
             '교육','디자인','사진','예술','영상','콘텐츠','문화','역사','뷰티','미용',
             '화장품','사회공헌','교류','언론','미디어','여행','호텔','항공','요리','식품',
             '의료','보건','정치','사회','법률','창업','자기계발','행사','페스티벌','환경',
             '에너지','기타','체육','헬스']                                                                  #위 키워드로 단어 정제
new_df = df.copy()                                                                                          #새로운 DataFrame 생성, 덮어쓰기 방지

for interest in interests:
    new_df['관심분야'] = new_df['관심분야'].apply(lambda x: '/'.join([re.sub(f'{interest}.+', interest, i) for i in x.split('/')]))
                                                                                                            # 위 '키워드'+@ 내용들 '키워드'로 교체 예)마케팅,마케팅창업,마케팅경제,마케팅행사 => 모두 '마케팅'으로 변경
df = new_df                                                                                                 # 새로운 DataFrame으로 기존의 DataFrame을 교체

interests_dummy = {'경영': '경영/컨설팅/마케팅', '컨설팅': '경영/컨설팅/마케팅', '마케팅': '경영/컨설팅/마케팅',
             '경제': '경제/금융', '금융': '경제/금융',
               'IT': '과학/공학/기술/IT', '과학': '과학/공학/기술/IT','공학':'과학/공학/기술/IT','기술':'과학/공학/기술/IT',
               '교육': '교육',
               '디자인': '디자인/사진/예술/영상/콘텐츠','사진':'디자인/사진/예술/영상/콘텐츠','예술':'디자인/사진/예술/영상/콘텐츠',
               '영상': '디자인/사진/예술/영상/콘텐츠','콘텐츠':'디자인/사진/예술/영상/콘텐츠',
               '문화':'문화/역사','역사':'문화/역사',
               '뷰티':'뷰티/미용/화장품','미용':'뷰티/미용/화장품','화장품':'뷰티/미용/화장품',
               '사회공헌':'사회공헌/교류','교류':'사회공헌/교류',
               '언론':'언론/미디어','미디어':'언론/미디어',
               '여행':'여행/호텔/항공','호텔':'여행/호텔/항공','항공':'여행/호텔/항공',
               '요리':'요리/식품','식품':'요리/식품',
               '의료':'의료/보건','보건':'의료/보건',
               '정치':'정치/사회/법률','사회':'정치/사회/법률','법률':'정치/사회/법률',
               '창업':'창업/자기계발','자기계발':'창업/자기계발',
               '행사':'행사/페스티벌','페스티벌':'행사/페스티벌',
               '환경':'환경/에너지','에너지':'환경/에너지',
               '기타':'기타/체육/헬스','체육':'기타/체육/헬스','헬스':'기타/체육/헬스'}

df['관심분야'] = df['관심분야'].apply(lambda x: interests_dummy.get(x, x))

interest_columns = pd.get_dummies(df['관심분야'], prefix='', prefix_sep='').fillna(0)

df = pd.concat([df, interest_columns], axis=1)



#우대역량 더미변수화
df["우대역량"] = df["우대역량"].str.replace(", 기타", "")                                                       #',기타'가 규칙없이 붙어있는경우가 많아서 삭제
df['우대역량'] = df['우대역량'].replace('기타', '우대역량_기타')                                                 #'기타'열이 관심분야에도 중복됨으로 교체 
df = df.join(df["우대역량"].str.split(', ', expand=True).stack().str.get_dummies().groupby(level=0).sum())

#모집인원
df["모집 인원"]=df["모집인원"].str.extract(r'(\d+)')     #숫자만 남기기
df=df.replace({"모집 인원":'000'},'999')                #000은 세자리수를 의미함으로 999명으로 변경
df=df.replace({"모집 인원":'00'},'99')                  #00은 두자리수를 의미함으로 99명으로 변경
df=df.replace({"모집 인원":'0'},'9')                    #0은 힌자리수를 의미함으로 9명으로 변경
df=df.replace({"모집 인원":np.NaN},0)                   #NaN(미정)값 0으로 변경
df=df.astype({"모집 인원":'int'})                   
student_bins=[0,1,21,101,50000]                        #범주화  0:인원미정/1-20(명):소/21-100(명):중/101-50000(명):대
student_scale=["인원미정","소","중","대"]               
df = pd.concat([df, pd.cut(df["모집 인원"], bins=student_bins, right=False, labels=student_scale).rename("모집인원규모")], axis=1) #분포범주화

#열삭제
df=df.drop(["대외활동명","접수기간","활동지역","활동기간"],axis='columns')

#데이터저장
df=df.to_csv("linkareer37.csv",encoding = 'euc-kr')


In [90]:
df = pd.read_csv('./list 2.csv',sep=',')
df = df.dropna()
interests_dummy = {'경영': '경영/컨설팅/마케팅', '컨설팅': '경영/컨설팅/마케팅', '마케팅': '경영/컨설팅/마케팅',
             '경제': '경제/금융', '금융': '경제/금융',
               'IT': '과학/공학/기술/IT', '과학': '과학/공학/기술/IT','공학':'과학/공학/기술/IT','기술':'과학/공학/기술/IT',
               '교육': '교육',
               '디자인': '디자인/사진/예술/영상/콘텐츠','사진':'디자인/사진/예술/영상/콘텐츠','예술':'디자인/사진/예술/영상/콘텐츠',
               '영상': '디자인/사진/예술/영상/콘텐츠','콘텐츠':'디자인/사진/예술/영상/콘텐츠',
               '문화':'문화/역사','역사':'문화/역사',
               '뷰티':'뷰티/미용/화장품','미용':'뷰티/미용/화장품','화장품':'뷰티/미용/화장품',
               '사회공헌':'사회공헌/교류','교류':'사회공헌/교류',
               '언론':'언론/미디어','미디어':'언론/미디어',
               '여행':'여행/호텔/항공','호텔':'여행/호텔/항공','항공':'여행/호텔/항공',
               '요리':'요리/식품','식품':'요리/식품',
               '의료':'의료/보건','보건':'의료/보건',
               '정치':'정치/사회/법률','사회':'정치/사회/법률','법률':'정치/사회/법률',
               '창업':'창업/자기계발','자기계발':'창업/자기계발',
               '행사':'행사/페스티벌','페스티벌':'행사/페스티벌',
               '환경':'환경/에너지','에너지':'환경/에너지',
               '기타':'기타/체육/헬스','체육':'기타/체육/헬스','헬스':'기타/체육/헬스'}

df['관심분야'] = df['관심분야'].apply(lambda x: interests_dummy.get(x, x))
df=df.to_csv("linkareer35.csv",encoding = 'euc-kr')

UnicodeEncodeError: 'euc_kr' codec can't encode character '\ufeff' in position 18: illegal multibyte sequence

In [10]:
result = df[df['관심분야'].str.contains('경제')]
print(result)
result.to_csv('example.txt', sep='\t', index=False)

       인덱스            기업명  모집인원  \
3    53991         관세청      00명   
38   54120        kmac       0명   
67   54177    골든트리투자자문      00명   
71   54227  아시아경제공동체재단      12명   
75   54237     복합금융컨설팅      20명   
151  54475         KB금융그룹  130명   
206  54623  (주)복합금융컨설팅      30명   
236  54722  복합금융컨설팅(주)      00명   
238  54725       라이징팝스      10명   
250  54748        한국수출입은행   15명   
287  54869           삼성생명   20명   
289  54877         EIC     100명   
291  54883           BC카드   00명   
300  54911         이상투자그룹    0명   
310  54954      복합금융컨설      80명   

                                                  우대역량             관심분야  IT  \
3                            사진/영상/디자인 툴, 취재/인터뷰/기사 경험          경제/금융기타   0   
38                       파워블로거/SNS, 외국어, 휴학 중/시간 투자 가능       문화/역사경제/금융   0   
67                                            모두 지원 가능          경제/금융기타   0   
71   사진/영상/디자인 툴, 콘텐츠 기획/제작 경험, 외국어, 타 대외활동 경험, 행사 ...            경제/금융   0   
75                                          

In [13]:
df['관심분야'] = df['관심분야'].str.replace('IT.+', 'IT', regex=True)
df

,인덱스,기업명,모집인원,우대역량,관심분야,IT,IT기타,IT창업,IT콘텐츠,경영,...,우대역량_기타,취재/인터뷰/기사 경험,컴퓨터 활용능력,콘텐츠 기획/제작 경험,타 대외활동 경험,파워블로거/SNS,행사 기획 경험,휴학 중/시간 투자 가능,모집 인원,모집인원규모
0,53987,스카이(SKY),00명,"파워블로거/SNS, 사진/영상/디자인 툴, 취재/인터뷰/기사 경험, 콘텐츠 기획/제...",과학/공학/기술/IT,0,0,0,1,0,...,0,1,0,1,1,1,0,0,99,중
1,53989,서주제과(주),00명,모두 지원 가능,요리/식품,0,0,0,0,0,...,0,0,0,0,0,0,0,0,99,중
2,53990,이석영뉴미디어도서관,0명,모두 지원 가능,문화/역사경영/컨설팅/마케팅,0,0,0,0,0,...,0,0,0,0,0,0,0,0,9,소
3,53991,관세청,00명,"사진/영상/디자인 툴, 취재/인터뷰/기사 경험",경제/금융,0,0,0,0,0,...,0,1,0,0,0,0,0,0,99,중
4,53993,독산동우시장일대 도시재생지원센터,00명,모두 지원 가능,문화/역사디자인/사진/예술/영상,0,0,0,0,0,...,0,0,0,0,0,0,0,0,99,중
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,55081,충청북도,6명,타 대외활동 경험,행사/페스티벌경영/컨설팅/마케팅,0,0,0,0,0,...,0,0,0,0,1,0,0,0,6,소
359,55083,아리얼,0명,파워블로거/SNS,경영/컨설팅/마케팅뷰티/미용/화장품,0,0,0,0,1,...,0,0,0,0,0,1,0,0,9,소
360,55084,듀이듀이,20명,모두 지원 가능,경영/컨설팅/마케팅뷰티/미용/화장품,0,0,0,0,1,...,0,0,0,0,0,0,0,0,20,소
361,55085,북센스출판사,10명,모두 지원 가능,경영/컨설팅/마케팅콘텐츠,0,0,0,0,1,...,0,0,0,0,0,0,0,0,10,소


In [9]:
print("모집인원 기초통계량",df["모집 인원"].describe())


모집인원 기초통계량 count     362.00000
mean       44.21547
std        67.06414
min         1.00000
25%         9.00000
50%        20.00000
75%        99.00000
max      1000.00000
Name: 모집 인원, dtype: float64
